In [1]:
import sqlite3
import pandas as pd
import psycopg2

In [2]:
titanic = pd.read_csv('titanic.csv')

In [3]:
print(titanic.shape)
titanic.head()

(887, 8)


,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [5]:
conn = psycopg2.connect(dbname='ciiiryug', user='ciiiryug',
                       password='K4JaUZIyW0gVjKm2zOh7UvuyEzr2qfzc', host='raja.db.elephantsql.com')

In [6]:
cursor = conn.cursor()

In [7]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
Survived                   887 non-null int64
Pclass                     887 non-null int64
Name                       887 non-null object
Sex                        887 non-null object
Age                        887 non-null float64
Siblings/Spouses Aboard    887 non-null int64
Parents/Children Aboard    887 non-null int64
Fare                       887 non-null float64
dtypes: float64(2), int64(4), object(2)
memory usage: 55.5+ KB


In [23]:
# Didn't work

# cursor.execute("""
#     CREATE TABLE titanic_data(
#     id integer PRIMARY KEY,
#     Survived integer,
#     Pclass integer,
#     Name text,
#     Sex text,
#     Age float,
#     SiblingsSpouses integer,
#     ParentsChildren integer,
#     Fare float)
#     """)

In [20]:
# Changing CSV data for Postgres

titanic['Survived'] = titanic['Survived'].replace({0:'no', 1:'yes'})
titanic['Pclass'] = titanic['Pclass'].replace({1:'first', 2:'second', 3:'third'})
titanic['Name'] = titanic['Name'].str.replace(r"[\"',]", '')

In [22]:
# Creating SQL Titanic file

conn = sqlite3.connect('titanic.sqlite3')
titanic.to_sql(name='passengers', con=conn, if_exists='replace')
cursor = conn.cursor()

/Users/mattmastin/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [ ]:
# Typing Paul's code from slack as exercise; looked better than mine
# Obviously not a working .py file

def create_db():
    db = ''
    user = ''
    password = ''
    host = ''
    
    conn = pyscopg2.connect(dbname=db, user=user,
                           password=password, host=host)
    
    cursor = conn.cursor()
    
    create_table = '''
    DROP TABLE titanic;
    CREATE TYPE pclass1 AS ENUM ('1', '2', '3');
    CREATE TABLE titanic (
    id SERIAL PRIMARY KEY,
    survival BOOL NOT NULL,
    pclass pclass1,
    name VARCHAR(255) NOT NULL,
    age FLOAT,
    sibsp INT,
    parch INT,
    fare FLOAT
    );
    '''
    # Pclass should be enumerated?
    
    cursor.execute(create_table)
    conn.commit()
    return conn

def csv_to_db(file, conn):
    cursor = conn.cursor()
    with open(file) as f:
        lines = f.readlines()[1:]
        for line in lines:
            survival, pclass1, name, sex, age, sipsp, parch, fare = line.split(',')
            cursor.execute("INSERT INTO titanic (survival, pclass, name, sex, age, sipsp, parch, fare)\
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, );"(surv,
                                                        str(pclass1),
                                                        str(name),
                                                        sex,
                                                        age,
                                                        sipsp,
                                                        parch,
                                                        fare
                                                        ))
    conn.commit()
    curser = conn.cursor()
    cursor.execute('''SELECT * FROM titanic''')
    print(f'return from equery: {cursor.fetchall()}\nSuccess!')
    
def main():
    conn = create_db()
    csv_to_db('titanic.csv', conn)
    
if __name__ = '__main__':
    main()

In [ ]:
# Practicing a different approach found

import sys

def pg_load_table(file_path, table_name, dbname, host, port, user, pwd):
    # Function to upload csv to target table
    try:
        conn = psycopg2.connect(dbname=dbname, host=host, port=port,\
                               user=user, password=pwd)
        print('connecting to database')
        cursor = conn.cursor()
        f = open(file_path, 'r')
        # truncate the table first
        cursor.execute('truncate {} cascade:'.format(table_name))
        print('truncated{}'.format(table_name))
        # load table from the file with header
        cursor.copy_expert('copy {} from STDIN CSV HEADER QUOTE\''.format(table_name), f)
        cursor.execute('commit;')
        print('loaded data into {}'.format(table_name))
        conn.close()
        print('DB connection closed')
        
    except Exception as e:
        print('error: {}'.format(str(e)))
        sys.exit(1)

# execution example
file_path = 'titanic.csv'
table_name = 'titanic'
dbname = ''
host = ''
port = ''
user = 'username'
pwd = 'password'
pg_load_table(file_path, table_name, dbname, host, port, user, pwd)